# Calico

## Setup Minikube

In [30]:
import subprocess
import yaml
from typing import List
def get_checks(gh_actions_file: str):
    """Get checks from a GitHub Actions file"""
    with open(gh_actions_file) as f:
        actions = yaml.safe_load(f)
    steps = actions['jobs']['scavenger']['steps']
    checks = [step for step in steps if 'Check ' in step['name']]
    return checks

def run_check(check):
    # cmd = f"cd /workspaces/intro-to-kube\n{check['run']}"
    cmd = check['run']
    subprocess.run(
        cmd, 
        capture_output=False, 
        shell=True, 
        executable='/bin/bash',
        cwd = '/workspaces/intro-to-kube'
    )

def run_checks(checks: List):
    """Run checks"""
    for check in checks:
        run_check(check)
        
checks = get_checks('/workspaces/intro-to-kube/.github/workflows/calico.yml')

In [31]:
!minikube start --network-plugin=cni --cni=calico

😄  minikube v1.31.2 on Ubuntu 20.04 (docker/amd64)
✨  Automatically selected the docker driver. Other choices: none, ssh
❗  With --network-plugin=cni, you will need to provide your own CNI. See --cni flag as a user-friendly alternative
📌  Using Docker driver with root privileges
👍  Starting control plane node minikube in cluster minikube
🚜  Pulling base image ...
🔥  Creating docker container (CPUs=2, Memory=2200MB) ...
🐳  Preparing Kubernetes v1.27.4 on Docker 24.0.4 ...
    ▪ Generating certificates and keys ...
    ▪ Booting up control plane ...
    ▪ Configuring RBAC rules ...
🔗  Configuring Calico (Container Networking Interface) ...
    ▪ Using image gcr.io/k8s-minikube/storage-provisioner:v5
🔎  Verifying Kubernetes components...
🌟  Enabled addons: storage-provisioner, default-storageclass
🏄  Done! kubectl is now configured to use "minikube" cluster and "default" namespace by default


## Create Pods

In [32]:
%%bash
# Add sleep to wait for minikube to start
sleep 5

# Create nginx pod in namespace a
kubectl create ns namespace-a
kubectl config set-context --current --namespace=namespace-a
kubectl create -f ./k8s/nginx-pod.yml --namespace=namespace-a

# Create nginx pod in namespace b
kubectl create ns namespace-b
kubectl config set-context --current --namespace=namespace-b
kubectl create -f ./k8s/nginx-pod.yml --namespace=namespace-b

# Create some postgres pod inside namespace c
kubectl create ns namespace-c
kubectl config set-context --current --namespace=namespace-c
kubectl create -f ./k8s/postgres-pod.yml --namespace=namespace-c

namespace/namespace-a created
Context "minikube" modified.
pod/nginx created
namespace/namespace-b created
Context "minikube" modified.
pod/nginx created
namespace/namespace-c created
Context "minikube" modified.
pod/postgres created


Let us wait till all pods are running

In [60]:
%%bash

# Define the list of pod names and namespaces to check
pod_names=("nginx" "nginx" "postgres")
namespaces=("namespace-a" "namespace-b" "namespace-c")

# Loop until all pods are running
while true; do
  # Check if all pods are running
  all_running=true
  for i in "${!pod_names[@]}"; do
    if ! kubectl get pod "${pod_names[$i]}" -n "${namespaces[$i]}" -o jsonpath='{.status.phase}' | grep -q "Running"; then
      all_running=false
      break
    fi
  done

  # If all pods are running, exit the loop
  if "$all_running"; then
    break
  fi

  # Wait for a few seconds before checking again
  sleep 5
done

In [61]:
!kubectl get pods -o wide -A

NAMESPACE     NAME                                       READY   STATUS    RESTARTS      AGE     IP              NODE       NOMINATED NODE   READINESS GATES
kube-system   calico-kube-controllers-85578c44bf-rsftw   1/1     Running   1 (13m ago)   14m     10.244.120.66   minikube   <none>           <none>
kube-system   calico-node-9gcq2                          1/1     Running   0             14m     192.168.49.2    minikube   <none>           <none>
kube-system   coredns-5d78c9869d-5dzx7                   1/1     Running   1 (13m ago)   14m     10.244.120.65   minikube   <none>           <none>
kube-system   etcd-minikube                              1/1     Running   0             14m     192.168.49.2    minikube   <none>           <none>
kube-system   kube-apiserver-minikube                    1/1     Running   0             14m     192.168.49.2    minikube   <none>           <none>
kube-system   kube-controller-manager-minikube           1/1     Running   0             14m     192.16

In [52]:
# Install curl in postgres pod
!kubectl exec -it postgres -n namespace-c -- su -c "apt-get -y update && apt-get -y install curl" root 2>/dev/null

Get:1 http://deb.debian.org/debian bookworm InRelease [151 kB]
Get:2 http://deb.debian.org/debian bookworm-updates InRelease [52.1 kB]        
Get:3 http://deb.debian.org/debian-security bookworm-security InRelease [48.0 kB]
Get:4 http://deb.debian.org/debian bookworm/main amd64 Packages [8,780 kB]     
Get:5 http://apt.postgresql.org/pub/repos/apt bookworm-pgdg InRelease [123 kB]
Get:6 http://deb.debian.org/debian bookworm-updates/main amd64 Packages [6,408 B]
Get:7 http://deb.debian.org/debian-security bookworm-security/main amd64 Packages [79.7 kB]
Get:8 http://apt.postgresql.org/pub/repos/apt bookworm-pgdg/main amd64 Packages [296 kB]
Fetched 9,536 kB in 1s (6,737 kB/s)                       
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  ca-certificates libbrotli1 libcurl4 libnghttp2-14 libpsl5 librtmp1 libssh2-1
  publicsuffix
The following NEW pa

In [53]:
# Check curl version in postgres pod
!kubectl exec -it postgres -n namespace-c -- sh -c "curl --version" 2>/dev/null

curl 7.88.1 (x86_64-pc-linux-gnu) libcurl/7.88.1 OpenSSL/3.0.11 zlib/1.2.13 brotli/1.0.9 zstd/1.5.4 libidn2/2.3.3 libpsl/0.21.2 (+libidn2/2.3.3) libssh2/1.10.0 nghttp2/1.52.0 librtmp/2.3 OpenLDAP/2.5.13
Release-Date: 2023-02-20
Protocols: dict file ftp ftps gopher gophers http https imap imaps ldap ldaps mqtt pop3 pop3s rtmp rtsp scp sftp smb smbs smtp smtps telnet tftp
Features: alt-svc AsynchDNS brotli GSS-API HSTS HTTP2 HTTPS-proxy IDN IPv6 Kerberos Largefile libz NTLM NTLM_WB PSL SPNEGO SSL threadsafe TLS-SRP UnixSockets zstd


## Set Networking Policies

![networking](https://uplimit.com/_next/image?url=https%3A%2F%2Fuplimit-ugc.com%2Fstatic%2Fcourse%2Fkubernetes-managing-containers-at-scale%2Fassets%2Fclm9w6ld001hk12ad3lx6dequ%2Fdiagram.png&w=750&q=75)

### Namespace A

In [62]:
%%writefile network-policy-namespace-a.yml
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: network-policy-for-namespace-a
  namespace: namespace-a
spec:
  policyTypes: [Egress]
  egress:
  - to:
    - namespaceSelector:
        matchLabels:
          name: namespace-c

Overwriting network-policy-namespace-a.yml


In [63]:
%%bash
kubectl apply -f network-policy-namespace-a.yml
kubectl describe networkpolicy network-policy-for-namespace-a -n namespace-a

networkpolicy.networking.k8s.io/network-policy-for-namespace-a unchanged
Name:         network-policy-for-namespace-a
Namespace:    namespace-a
Created on:   2023-10-14 12:51:23 +0000 UTC
Labels:       <none>
Annotations:  <none>
Spec:
  PodSelector:     <none> (Allowing the specific traffic to all pods in this namespace)
  Not affecting ingress traffic
  Allowing egress traffic:
    To Port: <any> (traffic allowed to all ports)
    To:
      NamespaceSelector: name=namespace-c
  Policy Types: Egress


In [ ]:
%%bash

kubectl apply -f network-policy-namespace-a.yml
sleep 5

POD_B_PORT=80
POD_C_PORT=5432
POD_B_IP=`kubectl get pod --all_namespaces -o wide | grep namespace-b | grep -E -o  '([0-9]{1,3}\.){3}[0-9]{1,3}'`
POD_C_IP=`kubectl get pod --all_namespaces -o wide | grep namespace-c | grep -E -o  '([0-9]{1,3}\.){3}[0-9]{1,3}'`

# CONNECT_TO_B=`kubectl exec nginx -n namespace-a -- timeout 5s curl $POD_B_IP:$POD_B_PORT &>/dev/null; echo $?`
# if [ "$CONNECT_TO_B" == "0" ]; then
# echo "✅ Can ping pods in namespace-b"
# else
# echo "❌ Cannot ping pods in namespace-b"
# fi

C_CONNECT_OUTPUT=`kubectl exec nginx -n namespace-a -- apt-get update && sudo apt-get install curl -y && timeout 5s curl $POD_C_IP:$POD_C_PORT &>/dev/null; echo $?`
CONNECT_TO_C=`echo $C_CONNECT_OUTPUT | grep -oE '\w+$'`
if [ "$CONNECT_TO_C" == "52" ]; then
echo "✅ Can ping pods in namespace-c"
else
echo "❌ Cannot ping pods in namespace-c"
fi

### Namespace B

In [27]:
%%writefile network-policy-namespace-b.yml
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: network-policy-for-namespace-b
  namespace: namespace-b
spec:
  policyTypes: [Ingress, Egress]
  ingress:
  - from:
    - namespaceSelector:
        matchLabels:
          name: namespace-a
  egress:
  - to:
    - namespaceSelector:
        matchLabels:
          name: namespace-a

Overwriting network-policy-namespace-b.yml


### Namespace C

In [216]:
%%writefile network-policy-namespace-c.yml
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: network-policy-for-namespace-c
  namespace: namespace-c
spec:
  podSelector:
    matchLabels:
      name: postgres
  ingress:
    - from: 
      - namespaceSelector:
          matchLabels:
            kubernetes.io/metadata.name: namespace-a
        podSelector:
          matchLabels:
            name: nginx

Overwriting network-policy-namespace-c.yml


In [221]:
!kubectl apply -f network-policy-namespace-c.yml

networkpolicy.networking.k8s.io/network-policy-for-namespace-c configured


In [ ]:
%%bash
kubectl config use-context minikube
POD_PORT=80
POD_A_IP=`kubectl get pod --all_namespaces -o wide | grep namespace-a | grep -E -o  '([0-9]{1,3}\.){3}[0-9]{1,3}'`
POD_B_IP=`kubectl get pod --all_namespaces -o wide | grep namespace-b | grep -E -o  '([0-9]{1,3}\.){3}[0-9]{1,3}'`

CONNECT_TO_A=`kubectl exec postgres -n namespace-c -- timeout 5s curl $POD_A_IP:$POD_PORT &>/dev/null; echo $?`
if [ "$CONNECT_TO_A" != "0" ]; then
echo "✅ Cannot ping pods in namespace-a"
else
echo "❌ Can ping pods in namespace-a"
fi

CONNECT_TO_B=`kubectl exec postgres -n namespace-c -- timeout 5s curl $POD_B_IP:$POD_PORT &>/dev/null; echo $?`
if [ "$CONNECT_TO_B" != "0" ]; then
echo "✅ Cannot ping pods in namespace-b"
else
echo "❌ Can ping pods in namespace-b"
fi

## Teardown

In [65]:
!minikube stop
!minikube delete

✋  Stopping node "minikube"  ...
🛑  Powering off "minikube" via SSH ...
🛑  1 node stopped.
🔥  Deleting "minikube" in docker ...
🔥  Deleting container "minikube" ...
🔥  Removing /home/codespace/.minikube/machines/minikube ...
💀  Removed all traces of the "minikube" cluster.
